In [2]:
#Whole Numbers, 3 Targets
import pandas as pd
import numpy as np
import tensorflow
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import datetime as dt

In [3]:
walmart = pd.read_csv('walmartdata.csv')
walmart.head()

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68


In [4]:
walmart['Start_of_Week'] = pd.to_datetime(walmart['Start_of_Week'])
walmart = walmart.loc[ walmart['Start_of_Week'].dt.year < 2012 ]
walmart = walmart.reset_index()
walmart = walmart.drop(['index'], axis=1)
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4544,6392,6392,45,2011-12-03,2011-12-09,3.42,188.6545,8.523,B,118221,49,2011,8.0,No Holiday,957155.31
4545,6393,6393,45,2011-12-10,2011-12-16,3.41,188.7979,8.523,B,118221,50,2011,4.0,No Holiday,1078905.68
4546,6394,6394,45,2011-12-17,2011-12-23,3.39,188.9300,8.523,B,118221,51,2011,6.0,No Holiday,1521957.99
4547,6395,6395,45,2011-12-24,2011-12-30,3.39,189.0620,8.523,B,118221,52,2011,3.0,Christmas,869403.63


In [5]:
totalave = walmart.groupby(["Store"])["Weekly_Sales"].mean()
fuelave = walmart.groupby(["Store"])["Fuel_Price"].mean()
unemave = walmart.groupby(["Store"])["Unemployment"].mean()
cpiave = walmart.groupby(["Store"])["CPI"].mean()
tempave = walmart.groupby(["Store"])["Temperature_C"].mean()

In [6]:
for i in range(len(walmart)):
    walmart.loc[i, "Ave_Sales"] = totalave[walmart.loc[i, "Store"]]
    walmart.loc[i, "Ave_Fuel"] = fuelave[walmart.loc[i, "Store"]]
    walmart.loc[i, "Ave_Temp"] = tempave[walmart.loc[i, "Store"]]
    walmart.loc[i, "Ave_Unem"] = unemave[walmart.loc[i, "Store"]]
    walmart.loc[i, "Ave_CPI"] = cpiave[walmart.loc[i, "Store"]]
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales,Ave_Sales,Ave_Fuel,Ave_Temp,Ave_Unem,Ave_CPI
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90,1.542090e+06,3.078416,19.683168,7.833257,213.624653
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44,1.542090e+06,3.078416,19.683168,7.833257,213.624653
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17,1.542090e+06,3.078416,19.683168,7.833257,213.624653
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59,1.542090e+06,3.078416,19.683168,7.833257,213.624653
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68,1.542090e+06,3.078416,19.683168,7.833257,213.624653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4544,6392,6392,45,2011-12-03,2011-12-09,3.42,188.6545,8.523,B,118221,49,2011,8.0,No Holiday,957155.31,7.958989e+05,3.262178,13.663366,8.680713,184.354230
4545,6393,6393,45,2011-12-10,2011-12-16,3.41,188.7979,8.523,B,118221,50,2011,4.0,No Holiday,1078905.68,7.958989e+05,3.262178,13.663366,8.680713,184.354230
4546,6394,6394,45,2011-12-17,2011-12-23,3.39,188.9300,8.523,B,118221,51,2011,6.0,No Holiday,1521957.99,7.958989e+05,3.262178,13.663366,8.680713,184.354230
4547,6395,6395,45,2011-12-24,2011-12-30,3.39,189.0620,8.523,B,118221,52,2011,3.0,Christmas,869403.63,7.958989e+05,3.262178,13.663366,8.680713,184.354230


In [7]:
walmart['Fuel_Diff'] = (walmart['Fuel_Price'] - walmart["Ave_Fuel"])
walmart['Temp_Diff'] = (walmart['Temperature_C'] - walmart["Ave_Temp"])
walmart['Unem_Diff'] = (walmart['Unemployment'] - walmart["Ave_Unem"])
walmart['CPI_Diff'] = (walmart['CPI'] - walmart["Ave_CPI"])

In [8]:
walmart = walmart.filter(items=['Fuel_Diff', "CPI_Diff", "Unem_Diff", "Temp_Diff", "Weekly_Sales","Ave_Sales"])

In [9]:
walmart.loc[walmart['Weekly_Sales'] < (0.85 * walmart["Ave_Sales"]), 'Label'] = 'Below' 
walmart.loc[walmart['Weekly_Sales'] > (1.15 * walmart["Ave_Sales"]), 'Label'] = 'Above' 
walmart.loc[(walmart['Weekly_Sales'] > (0.85 * walmart["Ave_Sales"])) & (walmart['Weekly_Sales'] < (1.15 * walmart["Ave_Sales"])), 'Label'] = 'Average'

In [10]:
walmart = walmart.filter(items=['Fuel_Diff', "CPI_Diff", "Unem_Diff", "Temp_Diff", "Label"])

In [11]:
walmart["Label"].value_counts()

Average    3918
Above       386
Below       245
Name: Label, dtype: int64

In [12]:
X = walmart.drop("Label", axis=1)
y = walmart["Label"]
print(X.shape, y.shape)

(4549, 4) (4549,)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

# Scale and train x value datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model

LogisticRegression(random_state=42)

In [375]:
model.fit(X_train_scaled, encoded_y_train)

LogisticRegression(random_state=42)

In [376]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8709877821495133
Testing Data Score: 0.8664596273291926


In [377]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'solver': ['newton-cg', 'liblinear', 'sag', 'saga'],
             'max_iter':[200, 500, 1000]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [378]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.872, total=   0.1s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.872, total=   0.2s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=1, max_iter=200, solver=sag ...................................
[CV] .

[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] .

[CV]  C=10, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=10, max_iter=200, solver=newton-cg ............................
[CV]  C=10, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=newton-cg ............................
[CV]  C=10, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  

[CV]  C=50, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=50, max_iter=200, solver=newton-cg ............................
[CV]  C=50, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=50, max_iter=200, solver=newton-cg ............................
[CV]  C=50, max_iter=200, solver=newton-cg, score=0.872, total=   0.1s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed:    6.3s finished


GridSearchCV(estimator=LogisticRegression(random_state=42),
             param_grid={'C': [1, 5, 10, 50], 'max_iter': [200, 500, 1000],
                         'solver': ['newton-cg', 'liblinear', 'sag', 'saga']},
             verbose=3)

In [379]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'max_iter': 200, 'solver': 'newton-cg'}
0.8709878887351291


In [382]:
logreg = LogisticRegression(C=1, max_iter=200, solver= 'newton-cg', random_state=42)
logreg.fit(X_train_scaled, encoded_y_train)
print(f"Training Data Score: {logreg.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {logreg.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8709877821495133
Testing Data Score: 0.8664596273291926


In [25]:
target = walmart["Label"]
target_names = ["Above", "Average", "Below"]

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf = rf.fit(X_train_scaled, encoded_y_train)
rf.score(X_test_scaled, encoded_y_test)

0.9015817223198594

In [25]:
feature_names = walmart.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.3264132834336524, 'CPI'),
 (0.26873914849490466, 'Fuel_Price'),
 (0.2206793881382728, 'Temperature_C'),
 (0.18416817993317022, 'Unemployment')]

In [385]:
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
             'class_weight': ['None', 'balanced']}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [386]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=random ..............
[CV]  class_weight=None, criterion=gini, splitter=random, score=nan, total=   0.0s
[CV] class_weight=None, cr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 199, in fit
    expanded_class_weight = compute_sample_weight(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/Users/ronal

[CV]  class_weight=balanced, criterion=entropy, splitter=best, score=0.851, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.827, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.836, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.839, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['None', 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [387]:
print(grid.best_params_)
print(grid.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'splitter': 'best'}
0.8482092706422512


In [18]:
ranfor = RandomForestClassifier(n_estimators=1000, random_state=42)
ranfor = ranfor.fit(X_train_scaled, encoded_y_train)
ranfor.score(X_test_scaled, encoded_y_test)

0.9015817223198594

In [19]:
clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train_scaled, encoded_y_train)
clf.score(X_test_scaled, encoded_y_test)

0.8699472759226714

In [20]:
feature_names = walmart.columns
sorted(zip(clf.feature_importances_, feature_names), reverse=True)

[(0.29307083248914667, 'Fuel_Diff'),
 (0.2771314624156992, 'CPI_Diff'),
 (0.26755301373374113, 'Temp_Diff'),
 (0.16224469136141312, 'Unem_Diff')]

In [391]:
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
             'class_weight': ['None', 'balanced']}
grid = GridSearchCV(clf, param_grid, verbose=3)

In [392]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=random ..............
[CV]  class_weight=None, criterion=gini, splitter=random, score=nan, total=   0.0s
[CV] class_weight=None, cr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 199, in fit
    expanded_class_weight = compute_sample_weight(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/Users/ronal

[CV]  class_weight=balanced, criterion=entropy, splitter=best, score=0.851, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.827, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.836, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.839, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['None', 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [393]:
print(grid.best_params_)
print(grid.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'splitter': 'best'}
0.8482092706422512


In [21]:
dectre = tree.DecisionTreeClassifier(random_state=42)
dectre = dectre.fit(X_train_scaled, encoded_y_train)
dectre.score(X_test_scaled, encoded_y_test)

0.8699472759226714

In [22]:
dectre_predictions = dectre.predict(X_test_scaled)

In [23]:
ranfor_predictions = ranfor.predict(X_test_scaled)

In [26]:
print(classification_report(encoded_y_test, ranfor_predictions,
                            target_names= target_names))

              precision    recall  f1-score   support

       Above       0.68      0.62      0.65        99
     Average       0.93      0.96      0.94       990
       Below       0.56      0.37      0.44        49

    accuracy                           0.90      1138
   macro avg       0.72      0.65      0.68      1138
weighted avg       0.89      0.90      0.90      1138



In [27]:
print(classification_report(encoded_y_test, dectre_predictions,
                            target_names= target_names))

              precision    recall  f1-score   support

       Above       0.57      0.70      0.63        99
     Average       0.94      0.91      0.92       990
       Below       0.35      0.45      0.40        49

    accuracy                           0.87      1138
   macro avg       0.62      0.68      0.65      1138
weighted avg       0.88      0.87      0.88      1138



In [43]:
predix = pd.read_csv('2012_predictions.csv')
predix = predix.drop(['Unnamed: 0'], axis=1)
predix.head()

,Date,Fuel_Price,Store,CPI,Unemployment,Temperature_C,Weekly_Sales
0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06
1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06
2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06
3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06
4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06


In [44]:
predix = predix.filter(items=['Date', 'Store', 'Fuel_Price', "CPI", "Unemployment", "Temperature_C"])

In [45]:
predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C
0,2012-01-01,1,3.130357,219.768684,7.768308,21.613198
1,2012-01-08,1,3.110403,219.979377,7.758391,21.654107
2,2012-01-15,1,3.090448,220.190070,7.748474,21.695015
3,2012-01-22,1,3.070493,220.400763,7.738557,21.735924
4,2012-01-29,1,3.050539,220.611456,7.728640,21.776833
...,...,...,...,...,...,...
2380,2012-12-02,45,2.543061,195.826121,8.124308,19.193262
2381,2012-12-09,45,2.525548,195.963795,8.116957,19.251467
2382,2012-12-16,45,2.508035,196.101468,8.109606,19.309672
2383,2012-12-23,45,2.490522,196.239142,8.102255,19.367877


In [46]:
predix
X = predix.drop("Date", axis=1)
X = X.drop("Store", axis=1)
X

,Fuel_Price,CPI,Unemployment,Temperature_C
0,3.130357,219.768684,7.768308,21.613198
1,3.110403,219.979377,7.758391,21.654107
2,3.090448,220.190070,7.748474,21.695015
3,3.070493,220.400763,7.738557,21.735924
4,3.050539,220.611456,7.728640,21.776833
...,...,...,...,...
2380,2.543061,195.826121,8.124308,19.193262
2381,2.525548,195.963795,8.116957,19.251467
2382,2.508035,196.101468,8.109606,19.309672
2383,2.490522,196.239142,8.102255,19.367877


In [47]:
predix_scaled = X_scaler.transform(X)

In [48]:
ranfor_labels = ranfor.predict(predix_scaled)

In [49]:
ranfor_labels = label_encoder.inverse_transform(ranfor_labels)
ranfor_labels

array(['Average', 'Average', 'Average', ..., 'Average', 'Average',
       'Average'], dtype=object)

In [50]:
ranfor_predix = predix
ranfor_predix["Label"] = ranfor_labels
ranfor_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-01-01,1,3.130357,219.768684,7.768308,21.613198,Average
1,2012-01-08,1,3.110403,219.979377,7.758391,21.654107,Average
2,2012-01-15,1,3.090448,220.190070,7.748474,21.695015,Average
3,2012-01-22,1,3.070493,220.400763,7.738557,21.735924,Average
4,2012-01-29,1,3.050539,220.611456,7.728640,21.776833,Average
...,...,...,...,...,...,...,...
2380,2012-12-02,45,2.543061,195.826121,8.124308,19.193262,Average
2381,2012-12-09,45,2.525548,195.963795,8.116957,19.251467,Average
2382,2012-12-16,45,2.508035,196.101468,8.109606,19.309672,Average
2383,2012-12-23,45,2.490522,196.239142,8.102255,19.367877,Average


In [51]:
ranfor_predix["Label"].value_counts()

Average    2385
Name: Label, dtype: int64

In [52]:
dectre_labels = dectre.predict(predix_scaled)

In [53]:
dectre_labels = label_encoder.inverse_transform(dectre_labels)
dectre_labels

array(['Above', 'Above', 'Above', ..., 'Above', 'Above', 'Above'],
      dtype=object)

In [54]:
dectre_predix = predix
dectre_predix["Label"] = dectre_labels
dectre_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-01-01,1,3.130357,219.768684,7.768308,21.613198,Above
1,2012-01-08,1,3.110403,219.979377,7.758391,21.654107,Above
2,2012-01-15,1,3.090448,220.190070,7.748474,21.695015,Above
3,2012-01-22,1,3.070493,220.400763,7.738557,21.735924,Above
4,2012-01-29,1,3.050539,220.611456,7.728640,21.776833,Above
...,...,...,...,...,...,...,...
2380,2012-12-02,45,2.543061,195.826121,8.124308,19.193262,Above
2381,2012-12-09,45,2.525548,195.963795,8.116957,19.251467,Above
2382,2012-12-16,45,2.508035,196.101468,8.109606,19.309672,Above
2383,2012-12-23,45,2.490522,196.239142,8.102255,19.367877,Above


In [55]:
dectre_predix["Label"].value_counts()

Above    2385
Name: Label, dtype: int64

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-01-01,1,3.130357,219.768684,7.768308,21.613198,Below
1,2012-01-08,1,3.110403,219.979377,7.758391,21.654107,Below
2,2012-01-15,1,3.090448,220.190070,7.748474,21.695015,Below
3,2012-01-22,1,3.070493,220.400763,7.738557,21.735924,Below
4,2012-01-29,1,3.050539,220.611456,7.728640,21.776833,Below
...,...,...,...,...,...,...,...
2380,2012-12-02,45,2.543061,195.826121,8.124308,19.193262,Average
2381,2012-12-09,45,2.525548,195.963795,8.116957,19.251467,Average
2382,2012-12-16,45,2.508035,196.101468,8.109606,19.309672,Average
2383,2012-12-23,45,2.490522,196.239142,8.102255,19.367877,Average


In [67]:
pre2012 = pd.read_csv('2012_predictions.csv')

In [68]:
for i in range(len(pre2012)):
    pre2012.loc[i, "Ave_Sales"] = totalave[pre2012.loc[i, "Store"]]
pre2012

,Unnamed: 0,Date,Fuel_Price,Store,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Sales
0,0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06,1.542090e+06
1,1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.542090e+06
2,2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.542090e+06
3,3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.542090e+06
4,4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.542090e+06
...,...,...,...,...,...,...,...,...,...
2380,2468,2012-12-02,2.543061,45,195.826121,8.124308,19.193262,8.700074e+05,7.958989e+05
2381,2469,2012-12-09,2.525548,45,195.963795,8.116957,19.251467,8.708043e+05,7.958989e+05
2382,2470,2012-12-16,2.508035,45,196.101468,8.109606,19.309672,8.716012e+05,7.958989e+05
2383,2471,2012-12-23,2.490522,45,196.239142,8.102255,19.367877,8.723981e+05,7.958989e+05


In [69]:
pre2012.loc[pre2012['Weekly_Sales'] < (0.85 * pre2012["Ave_Sales"]), 'Label'] = 'Below' 
pre2012.loc[pre2012['Weekly_Sales'] > (1.15 * pre2012["Ave_Sales"]), 'Label'] = 'Above' 
pre2012.loc[(pre2012['Weekly_Sales'] > (0.85 * pre2012["Ave_Sales"])) & (pre2012['Weekly_Sales'] < (1.15 * pre2012["Ave_Sales"])), 'Label'] = 'Average'
proph_predix = pre2012

In [70]:
proph_predix["Label"].value_counts()

Average    2005
Above       274
Below       106
Name: Label, dtype: int64

In [71]:
proph_predix = proph_predix.rename(columns={"Label": "Proph_Label"})

In [72]:
dectre_predix["Label"].value_counts()

Average    2002
Below       276
Above       107
Name: Label, dtype: int64

In [73]:
dectre_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-01-01,1,3.130357,219.768684,7.768308,21.613198,Below
1,2012-01-08,1,3.110403,219.979377,7.758391,21.654107,Below
2,2012-01-15,1,3.090448,220.190070,7.748474,21.695015,Below
3,2012-01-22,1,3.070493,220.400763,7.738557,21.735924,Below
4,2012-01-29,1,3.050539,220.611456,7.728640,21.776833,Below
...,...,...,...,...,...,...,...
2380,2012-12-02,45,2.543061,195.826121,8.124308,19.193262,Average
2381,2012-12-09,45,2.525548,195.963795,8.116957,19.251467,Average
2382,2012-12-16,45,2.508035,196.101468,8.109606,19.309672,Average
2383,2012-12-23,45,2.490522,196.239142,8.102255,19.367877,Average


In [79]:
ranfor_predix["Label"].value_counts()

Average    2435
Below        22
Above        18
Name: Label, dtype: int64

In [74]:
final = pd.merge(proph_predix, dectre_predix, how="outer", on=["Date", "Store", "Fuel_Price", "CPI", "Unemployment", "Temperature_C"])


In [75]:
final = final.drop(['Unnamed: 0'], axis=1)
final

,Date,Fuel_Price,Store,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Sales,Proph_Label,Label
0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06,1.542090e+06,Average,Below
1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.542090e+06,Average,Below
2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.542090e+06,Average,Below
3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.542090e+06,Average,Below
4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.542090e+06,Average,Below
...,...,...,...,...,...,...,...,...,...,...
2380,2012-12-02,2.543061,45,195.826121,8.124308,19.193262,8.700074e+05,7.958989e+05,Average,Average
2381,2012-12-09,2.525548,45,195.963795,8.116957,19.251467,8.708043e+05,7.958989e+05,Average,Average
2382,2012-12-16,2.508035,45,196.101468,8.109606,19.309672,8.716012e+05,7.958989e+05,Average,Average
2383,2012-12-23,2.490522,45,196.239142,8.102255,19.367877,8.723981e+05,7.958989e+05,Average,Average


In [76]:
final = final.rename(columns={"Fuel_Price": "Pred_Fuel_Pric", "CPI": "Pred_CPI", "Unemployment": "Pred_Unem", "Temperature_C": "Pred_Temp", 'Weekly_Sales': 'Pred_Week_Sales'})
final

,Date,Pred_Fuel_Pric,Store,Pred_CPI,Pred_Unem,Pred_Temp,Pred_Week_Sales,Ave_Sales,Proph_Label,Label
0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06,1.542090e+06,Average,Below
1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.542090e+06,Average,Below
2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.542090e+06,Average,Below
3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.542090e+06,Average,Below
4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.542090e+06,Average,Below
...,...,...,...,...,...,...,...,...,...,...
2380,2012-12-02,2.543061,45,195.826121,8.124308,19.193262,8.700074e+05,7.958989e+05,Average,Average
2381,2012-12-09,2.525548,45,195.963795,8.116957,19.251467,8.708043e+05,7.958989e+05,Average,Average
2382,2012-12-16,2.508035,45,196.101468,8.109606,19.309672,8.716012e+05,7.958989e+05,Average,Average
2383,2012-12-23,2.490522,45,196.239142,8.102255,19.367877,8.723981e+05,7.958989e+05,Average,Average


In [77]:
walmart

,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2.57,211.0964,8.106,6.0,Average
1,2.55,211.2422,8.106,4.0,Average
2,2.51,211.2891,8.106,4.0,Average
3,2.56,211.3196,8.106,8.0,Average
4,2.62,211.3501,8.106,8.0,Average
...,...,...,...,...,...
4544,3.42,188.6545,8.523,8.0,Above
4545,3.41,188.7979,8.523,4.0,Above
4546,3.39,188.9300,8.523,6.0,Above
4547,3.39,189.0620,8.523,3.0,Average


In [91]:
walmart = pd.read_csv('walmartdata.csv')
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6434,6434,6434,45,2012-09-22,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95
6435,6435,6435,45,2012-09-29,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07
6436,6436,6436,45,2012-10-06,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36
6437,6437,6437,45,2012-10-13,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53


In [92]:
walmart['Start_of_Week'] = pd.to_datetime(walmart['Start_of_Week'])
walmart = walmart.loc[ walmart['Start_of_Week'].dt.year == 2012 ]
walmart = walmart.reset_index()
walmart = walmart.drop(['index'], axis=1)
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,101,101,1,2012-01-07,2012-01-13,3.26,219.8925,7.348,A,151315,2,2012,9.0,No Holiday,1459601.17
1,102,102,1,2012-01-14,2012-01-20,3.27,219.9857,7.348,A,151315,3,2012,12.0,No Holiday,1394393.84
2,103,103,1,2012-01-21,2012-01-27,3.29,220.0789,7.348,A,151315,4,2012,12.0,No Holiday,1319325.59
3,104,104,1,2012-01-28,2012-02-03,3.36,220.1720,7.348,A,151315,5,2012,14.0,No Holiday,1636339.65
4,105,105,1,2012-02-04,2012-02-10,3.41,220.2652,7.348,A,151315,6,2012,9.0,No Holiday,1802477.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,6434,6434,45,2012-09-22,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95
1886,6435,6435,45,2012-09-29,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07
1887,6436,6436,45,2012-10-06,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36
1888,6437,6437,45,2012-10-13,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53


In [93]:
for i in range(len(walmart)):
    walmart.loc[i, "Start_of_Week"] += dt.timedelta(days=1)
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,101,101,1,2012-01-08,2012-01-13,3.26,219.8925,7.348,A,151315,2,2012,9.0,No Holiday,1459601.17
1,102,102,1,2012-01-15,2012-01-20,3.27,219.9857,7.348,A,151315,3,2012,12.0,No Holiday,1394393.84
2,103,103,1,2012-01-22,2012-01-27,3.29,220.0789,7.348,A,151315,4,2012,12.0,No Holiday,1319325.59
3,104,104,1,2012-01-29,2012-02-03,3.36,220.1720,7.348,A,151315,5,2012,14.0,No Holiday,1636339.65
4,105,105,1,2012-02-05,2012-02-10,3.41,220.2652,7.348,A,151315,6,2012,9.0,No Holiday,1802477.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,6434,6434,45,2012-09-23,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95
1886,6435,6435,45,2012-09-30,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07
1887,6436,6436,45,2012-10-07,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36
1888,6437,6437,45,2012-10-14,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53


In [101]:
final['Date'] = pd.to_datetime(final['Date'])
end_date = walmart['Start_of_Week'].iloc[[-1]].item()
before_end_date = final["Date"] <= end_date
final = final.loc[before_end_date]

final

,Date,Pred_Fuel_Pric,Store,Pred_CPI,Pred_Unem,Pred_Temp,Pred_Week_Sales,Ave_Sales,Proph_Label,Label
0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06,1.542090e+06,Average,Below
1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.542090e+06,Average,Below
2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.542090e+06,Average,Below
3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.542090e+06,Average,Below
4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.542090e+06,Average,Below
...,...,...,...,...,...,...,...,...,...,...
2370,2012-09-23,2.718193,45,194.449387,8.197820,18.611209,8.620383e+05,7.958989e+05,Average,Average
2371,2012-09-30,2.700680,45,194.587061,8.190469,18.669415,8.628352e+05,7.958989e+05,Average,Average
2372,2012-10-07,2.683167,45,194.724734,8.183118,18.727620,8.636321e+05,7.958989e+05,Average,Average
2373,2012-10-14,2.665653,45,194.862407,8.175767,18.785825,8.644290e+05,7.958989e+05,Average,Average


In [104]:
walmart = walmart.rename(columns={"Start_of_Week": "Date"})

In [114]:
comparison = pd.merge(final, walmart, how="outer", on=["Date", "Store"])
comparison = comparison.iloc[1:]
comparison

,Date,Pred_Fuel_Pric,Store,Pred_CPI,Pred_Unem,Pred_Temp,Pred_Week_Sales,Ave_Sales,Proph_Label,Label,...,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.542090e+06,Average,Below,...,3.26,219.8925,7.348,A,151315.0,2.0,2012.0,9.0,No Holiday,1459601.17
2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.542090e+06,Average,Below,...,3.27,219.9857,7.348,A,151315.0,3.0,2012.0,12.0,No Holiday,1394393.84
3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.542090e+06,Average,Below,...,3.29,220.0789,7.348,A,151315.0,4.0,2012.0,12.0,No Holiday,1319325.59
4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.542090e+06,Average,Below,...,3.36,220.1720,7.348,A,151315.0,5.0,2012.0,14.0,No Holiday,1636339.65
5,2012-02-05,3.030584,1,220.822149,7.718724,21.817742,1.611106e+06,1.542090e+06,Average,Below,...,3.41,220.2652,7.348,A,151315.0,6.0,2012.0,9.0,No Holiday,1802477.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,2012-09-23,2.718193,45,194.449387,8.197820,18.611209,8.620383e+05,7.958989e+05,Average,Average,...,4.00,192.0136,8.684,B,118221.0,39.0,2012.0,18.0,No Holiday,713173.95
1931,2012-09-30,2.700680,45,194.587061,8.190469,18.669415,8.628352e+05,7.958989e+05,Average,Average,...,3.98,192.1704,8.667,B,118221.0,40.0,2012.0,18.0,No Holiday,733455.07
1932,2012-10-07,2.683167,45,194.724734,8.183118,18.727620,8.636321e+05,7.958989e+05,Average,Average,...,4.00,192.3273,8.667,B,118221.0,41.0,2012.0,12.0,No Holiday,734464.36
1933,2012-10-14,2.665653,45,194.862407,8.175767,18.785825,8.644290e+05,7.958989e+05,Average,Average,...,3.97,192.3309,8.667,B,118221.0,42.0,2012.0,14.0,No Holiday,718125.53


In [115]:
comparison.columns

Index(['Date', 'Pred_Fuel_Pric', 'Store', 'Pred_CPI', 'Pred_Unem', 'Pred_Temp',
       'Pred_Week_Sales', 'Ave_Sales', 'Proph_Label', 'Label', 'Unnamed: 0',
       'ID', 'Week_Date', 'Fuel_Price', 'CPI', 'Unemployment', 'Type', 'Size',
       'Week', 'Year', 'Temperature_C', 'Holiday_Name', 'Weekly_Sales'],
      dtype='object')

In [116]:
comparison = comparison[["Date", "Store", "Fuel_Price", 'Pred_Fuel_Pric', 'CPI', 'Pred_CPI', 'Temperature_C', 'Pred_Temp', 'Unemployment', 'Pred_Unem', 'Weekly_Sales', "Pred_Week_Sales", 'Ave_Sales','Proph_Label', 'Label']]
comparison

,Date,Store,Fuel_Price,Pred_Fuel_Pric,CPI,Pred_CPI,Temperature_C,Pred_Temp,Unemployment,Pred_Unem,Weekly_Sales,Pred_Week_Sales,Ave_Sales,Proph_Label,Label
1,2012-01-08,1,3.26,3.110403,219.8925,219.979377,9.0,21.654107,7.348,7.758391,1459601.17,1.606048e+06,1.542090e+06,Average,Below
2,2012-01-15,1,3.27,3.090448,219.9857,220.190070,12.0,21.695015,7.348,7.748474,1394393.84,1.607312e+06,1.542090e+06,Average,Below
3,2012-01-22,1,3.29,3.070493,220.0789,220.400763,12.0,21.735924,7.348,7.738557,1319325.59,1.608577e+06,1.542090e+06,Average,Below
4,2012-01-29,1,3.36,3.050539,220.1720,220.611456,14.0,21.776833,7.348,7.728640,1636339.65,1.609841e+06,1.542090e+06,Average,Below
5,2012-02-05,1,3.41,3.030584,220.2652,220.822149,9.0,21.817742,7.348,7.718724,1802477.43,1.611106e+06,1.542090e+06,Average,Below
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,2012-09-23,45,4.00,2.718193,192.0136,194.449387,18.0,18.611209,8.684,8.197820,713173.95,8.620383e+05,7.958989e+05,Average,Average
1931,2012-09-30,45,3.98,2.700680,192.1704,194.587061,18.0,18.669415,8.667,8.190469,733455.07,8.628352e+05,7.958989e+05,Average,Average
1932,2012-10-07,45,4.00,2.683167,192.3273,194.724734,12.0,18.727620,8.667,8.183118,734464.36,8.636321e+05,7.958989e+05,Average,Average
1933,2012-10-14,45,3.97,2.665653,192.3309,194.862407,14.0,18.785825,8.667,8.175767,718125.53,8.644290e+05,7.958989e+05,Average,Average


In [118]:
comparison = comparison.rename(columns={"Label": "Model_Label"})

In [121]:

comparison.loc[comparison['Weekly_Sales'] < (0.85 * comparison["Ave_Sales"]), 'Real_Label'] = 'Below' 
comparison.loc[comparison['Weekly_Sales'] > (1.15 * comparison["Ave_Sales"]), 'Real_Label'] = 'Above' 
comparison.loc[(comparison['Weekly_Sales'] > (0.85 * comparison["Ave_Sales"])) & (comparison['Weekly_Sales'] < (1.15 * comparison["Ave_Sales"])), 'Real_Label'] = 'Average'

In [122]:
comparison

,Date,Store,Fuel_Price,Pred_Fuel_Pric,CPI,Pred_CPI,Temperature_C,Pred_Temp,Unemployment,Pred_Unem,Weekly_Sales,Pred_Week_Sales,Ave_Sales,Proph_Label,Model_Label,Real_Label
1,2012-01-08,1,3.26,3.110403,219.8925,219.979377,9.0,21.654107,7.348,7.758391,1459601.17,1.606048e+06,1.542090e+06,Average,Below,Average
2,2012-01-15,1,3.27,3.090448,219.9857,220.190070,12.0,21.695015,7.348,7.748474,1394393.84,1.607312e+06,1.542090e+06,Average,Below,Average
3,2012-01-22,1,3.29,3.070493,220.0789,220.400763,12.0,21.735924,7.348,7.738557,1319325.59,1.608577e+06,1.542090e+06,Average,Below,Average
4,2012-01-29,1,3.36,3.050539,220.1720,220.611456,14.0,21.776833,7.348,7.728640,1636339.65,1.609841e+06,1.542090e+06,Average,Below,Average
5,2012-02-05,1,3.41,3.030584,220.2652,220.822149,9.0,21.817742,7.348,7.718724,1802477.43,1.611106e+06,1.542090e+06,Average,Below,Above
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,2012-09-23,45,4.00,2.718193,192.0136,194.449387,18.0,18.611209,8.684,8.197820,713173.95,8.620383e+05,7.958989e+05,Average,Average,Average
1931,2012-09-30,45,3.98,2.700680,192.1704,194.587061,18.0,18.669415,8.667,8.190469,733455.07,8.628352e+05,7.958989e+05,Average,Average,Average
1932,2012-10-07,45,4.00,2.683167,192.3273,194.724734,12.0,18.727620,8.667,8.183118,734464.36,8.636321e+05,7.958989e+05,Average,Average,Average
1933,2012-10-14,45,3.97,2.665653,192.3309,194.862407,14.0,18.785825,8.667,8.175767,718125.53,8.644290e+05,7.958989e+05,Average,Average,Average
